<a href="https://colab.research.google.com/github/gpandu/BERT-Pretraining/blob/main/BERT_pre_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install tokenizers
!pip install datasets
!pip install transformers
!pip install apache_beam

  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.51.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.


  Using cached dill-0.3.1.1-py3-none-any.whl
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.15 requires dill>=0.3.7, but you have dill 0.3.1.1 which is incompatible.


In [1]:
import os; import psutil; import timeit
from datasets import load_dataset

mem_before = psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)
wiki = load_dataset("wikipedia", "20220301.en", split="train")
mem_after = psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)

print(f"RAM memory used: {(mem_after - mem_before)} MB")

RAM memory used: 64.6640625 MB


In [2]:
wiki.features

{'id': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [3]:
#from datasets import concatenate_datasets, load_dataset

#bookcorpus = load_dataset("bookcorpus", split="train", streaming=True)
tokenizer_batch_len = 1000
def get_training_corpus():
    for start_idx in range(0, len(wiki)/2, tokenizer_batch_len):
        samples = wiki[start_idx : start_idx + tokenizer_batch_len]
        yield samples["text"]

In [10]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    BertWordPieceTokenizer
)

from transformers import BertTokenizerFast

In [5]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
vocab_size = 30_522
max_length = 512
# whether to truncate
truncate_longer_samples = False

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [11]:
bert_tokenizer = tokenizer.train_new_from_iterator(text_iterator=get_training_corpus(), vocab_size=32_000)
bert_tokenizer.save_pretrained("tokenizer")

AttributeError: ignored

In [ ]:
# train the tokenizer
tokenizer.train_from_iterator(get_training_corpus(), vocab_size=vocab_size, special_tokens=special_tokens)
tokenizer.enable_truncation(max_length=max_length)

In [ ]:
from tqdm import tqdm
from transformers import BertTokenizerFast

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [ ]:
ids = tokenizer.encode("but just one look at a minion sent him practically catatonic .", "but just one look at a minion sent him practically catatonic .")
tokens = tokenizer.decode(ids)
tokens


'[CLS] but just one look at a minion sent him practically catatonic. [SEP] but just one look at a minion sent him practically catatonic. [SEP]'

In [ ]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
def group_texts(examples):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result


if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

In [ ]:
import tensorflow as tf

class BERT(tf.keras.Model):
  def __init__():
    print("init")
  def call():
    tf.keras.layers.


In [ ]:
# create a python generator to dynamically load the data
def batch_iterator(batch_size=10000):
    for i in tqdm(range(0, len(raw_datasets), batch_size)):
        yield raw_datasets[i : i + batch_size]["text"]

# create a tokenizer from existing one to re-use special tokens
tokenizer = BertTokenizerFast.train_new_from_iterator
